In [2]:
import pandas as pd


In [3]:
train = pd.read_csv('data/train.csv', sep=',', index_col=0,
                       dtype = {'row_id': 'int64', 'timestamp': 'int64', 'user_id': 'int32', 'content_id': 'int16', 
                                'content_type_id': 'int8', 'task_container_id': 'int16', 'user_answer': 'int8', 
                                'answered_correctly': 'int8', 'prior_question_elapsed_time': 'float32', 
                                'prior_question_had_explanation': 'boolean'
                               }
                      )

In [4]:
train.head()

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
row_id,,,,,,,,,
0,0,115,5692,0,1,3,1,NaN,<NA>
1,56943,115,5716,0,2,2,1,37000.0,False
2,118363,115,128,0,0,0,1,55000.0,False
3,131167,115,7860,0,3,0,1,19000.0,False
4,137965,115,7922,0,4,1,1,11000.0,False


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101230332 entries, 0 to 101230331
Data columns (total 9 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   timestamp                       int64  
 1   user_id                         int32  
 2   content_id                      int16  
 3   content_type_id                 int8   
 4   task_container_id               int16  
 5   user_answer                     int8   
 6   answered_correctly              int8   
 7   prior_question_elapsed_time     float32
 8   prior_question_had_explanation  boolean
dtypes: boolean(1), float32(1), int16(2), int32(1), int64(1), int8(3)
memory usage: 3.1 GB


In [ ]:
"""
Для удобства анализа приведем последний столбец к числовому типу
Сначала переведем пустые значения в False (в целях анализа эти значения равнозначны)
Затем конвертируем в числовой формат
"""
for i in range(train.shape[0]):
    if pd.isna(train['prior_question_had_explanation'][i]) == True:
        train['prior_question_had_explanation'][i] = False
train['prior_question_had_explanation'] = train['prior_question_had_explanation'].astype('int8')

In [ ]:
train.info()

In [ ]:
"""
Исследуем кол-во уникальных значений для разных колонок
"""
unique_list = []
for col in train.columns:
    item = (col, train[col].nunique(), train[col].dtype)
    unique_list.append(item)
unique_counts = pd.DataFrame(unique_list,
                             columns=['Column_Name', 'Num_Unique', 'Type']
                            ).sort_values(by='Num_Unique',  ignore_index=True)
display(unique_counts)

In [ ]:
"""
Исследуем кол-во уникальных значений для разных колонок
"""
unique_list = []
for col in train.columns:
    item = (col, train[col].nunique(), train[col].dtype)
    unique_list.append(item)
unique_counts = pd.DataFrame(unique_list,
                             columns=['Column_Name', 'Num_Unique', 'Type']
                            ).sort_values(by='Num_Unique',  ignore_index=True)
display(unique_counts)

In [ ]:
"""
Исследуем соотношение правильных и неправильных ответов (исключаем из рассмотрения лекции)
"""
train[train['answered_correctly'] != -1]['answered_correctly'].value_counts(normalize=True) 
train[train['answered_correctly'] != -1]['answered_correctly'].value_counts()[1] 
train[train['answered_correctly'] != -1]['answered_correctly'].value_counts()[0]
train[train['answered_correctly'] != -1]['answered_correctly'].mean()

# 66% составляют правильные ответы, 34% - неправильные
# всего 65244627 правильных ответов и 34026673 неправильных ответов
# 0.657 - средний балл студента

In [ ]:
"""
Посчитаем среднее время на решение вопроса
"""
train['prior_question_elapsed_time'].mean()

In [ ]:
"""
Приведем средние данные по общей базе данных:
"""
train[train['content_type_id'] == 0].shape[0] / train['user_id'].nunique()
train[train['answered_correctly'] != -1]['answered_correctly'].sum() / train['user_id'].nunique()
train[train['content_type_id'] == 1].shape[0] / train['user_id'].nunique()
train['prior_question_had_explanation'].sum() / train['user_id'].nunique()
train['prior_question_elapsed_time'].mean()

In [ ]:
"""
Посмотрим, есть ли разница в % правильных ответов в зависиомсти от:
- времени с начала сессии
- среднего времеми ответа студента на вопросы
- видел ли студент правильный ответ на предыдущее задание
"""
pd.DataFrame({'timestamp': train[train['answered_correctly'] != -1]['timestamp'].groupby(train['answered_correctly']).mean(),
              'prior_questions_time': train[train['answered_correctly'] != -1]['prior_question_elapsed_time'].groupby(train['answered_correctly']).mean(),
              'had_explanation': train[train['answered_correctly'] != -1]['prior_question_had_explanation'].groupby(train['answered_correctly']).sum()
             }
            )

# Можно сделать следующие выводы:
# - чем ближе к началу сессии вопрос, тем менее успешно его проходят студенты 
# (т.е. успешность коррелирует со временем, проведенным на платформе)
# - студенты, тратящие меньше времени на решение, чаще отвечают правильно
# - если студент видел объяснение предыдущий задачи, вероятность правильного ответа удваивается

In [ ]:
"""
Cоставим новый дф по уникальным id студентов
Cначала сформируем необходимые списки для df
"""
sudents_list = list(train['user_id'].unique())
users_q = len(sudents_list)

# Всего 393656 уникальных студентов

In [ ]:
"""
Поскольку число уникальных студентов слишком велико для быстрой обработки, 
возьмем из нее 500 уникальных и проанализируем их активность.
"""
sudents_list = sudents_list[:500]

In [ ]:
"""
Посчитаем время на платформе для каждого студента
"""
time = []
for student in sudents_list:
    t = train[train['user_id'] == student]['timestamp'].max()
    time.append(t)

In [ ]:
"""
Количество отвеченных вопросов
"""
ques_quant = []
for student in sudents_list:
    q = train[(train['user_id'] == student) & (train['content_type_id'] == 0)]['content_type_id'].count()
    ques_quant.append(q)

In [ ]:
"""
Cредний балл студента
"""
av_grade = []
for student in sudents_list:
    g = train[(train['user_id'] == student) & (train['answered_correctly'] != -1)]['answered_correctly'].mean()
    av_grade.append(g)
"""
Кол-во просмотренных лекций
"""
lec_watched = []
for student in sudents_list:
    l = train[train['user_id'] == student]['content_type_id'].sum()
    lec_watched.append(l)
"""
Кол-во виденных объяснений на предыдущие вопросы
"""
expl_watched = []
for student in sudents_list:
    e = train[train['user_id'] == student]['prior_question_had_explanation'].sum()
    expl_watched.append(e)
"""
Время, в среднем потраченное студентом на каждый вопрос
"""
ques_time = []
for student in sudents_list:
    qt = train[(train['user_id'] == student)]['prior_question_elapsed_time'].mean()
    ques_time.append(qt)
"""
Запишем данные по 500 отобранным студентам в единую таблицу
"""
students = pd.DataFrame({'user_id': sudents_list,
                        'time': time,
                        'ques_quant': ques_quant,
                        'av_grade': av_grade,
                        'lec_watched': lec_watched,
                        'expl_watched': expl_watched,
                         'ques_time': ques_time
                       }
                      )
students = students.astype({'user_id': 'int32',
                            'time': 'int64',
                            'ques_quant': 'int16', 
                            'av_grade': 'float32',
                            'lec_watched': 'int16',
                            'expl_watched': 'int16',
                            'ques_time': 'float32'
                           })

In [ ]:
students.head()

In [ ]:
"""
Посмотрим, насколько наша выборка похожа на общие данные:
"""
students.describe()

In [ ]:
"""
Гистограма распределения средних оценок среди студентов
"""
students['av_grade'].hist(bins=20)

In [ ]:
students['av_grade'].median()

In [ ]:
"""
Построим диаграмму рассеяния, показывающую зависимость правильных ответов от веремени, проведенного на платформе
Чтобы убрать выбросы, ограничим время 3000000.
"""
students[students['time'] < 3000000].plot.scatter(x='av_grade', y='time')

In [ ]:
"""
Посмотрим на гистограмме, в какой момент большинство студентов прекращает занятия
"""
students[students['time'] < 3000000].hist(column=['time'])

In [ ]:
"""
Построим диаграмму рассеяния для студентов, не бросивших учебу в течение 1го часа.
"""
students[students['time'] > 3000000].plot.scatter(x='av_grade', y='time')

In [ ]:
"""
Для наглядности проведем разбиение студентов по временным группам:
 - Добавим столбец с временной группировкой
 - Построим диаграмму размаха
"""

def time_convert(e):
    if e < students['time'].quantile(0.1): return 0
    elif e < students['time'].quantile(0.2): return 1
    elif e < students['time'].quantile(0.3): return 2
    elif e < students['time'].quantile(0.4): return 3
    elif e < students['time'].quantile(0.5): return 4
    elif e < students['time'].quantile(0.6): return 5
    elif e < students['time'].quantile(0.7): return 6
    elif e < students['time'].quantile(0.8): return 7
    elif e < students['time'].quantile(0.9): return 8
    else: return 9
    
students['time_group'] = students['time'].apply(time_convert)
students.boxplot(column=['av_grade'], by=['time_group'])

# Первый эффект от занятий становится виден, если студент не бросает заниматься
# При этом, если студент потратил на занятия не менее 55 часов, его оценка становится устойчиво выше средней
# и продолжает расти со временем.
# Однако, если курс не пройден за ~ полгода, оценка становится нестабильной
# Т.е. стоит не только стимулировать студентов не бросать занятия после первого подхода,
# но и закончить курс за первые полгода обучения.

In [ ]:
"""
Построим диаграмму рассеяния, показывающую зависимость правильных ответов от кол-ва сделанных заданий
Ограничим выборку студентами, не бросившими обучение сразу.
"""
students[(students['ques_quant'] < 1000) & (students['time'] > 3000000)].plot.scatter(x='av_grade', y='ques_quant', c='red')

# чтобы убрать выбросы, ограничим данные сверху 1000
# На графике довольно очевидна связь успеваемости с кол-вом отвеченных вопросов
# Стоит провести группировку вопросов по кол-ву для дальнейшего анализа

In [ ]:
"""
Построим новый столбец с группировкой по кол-ву отвеченных вопросов
Далее строим диаграмму размаха
"""
def ques_convert(e):
    if e < students['ques_quant'].quantile(0.1): return 0
    elif e < students['ques_quant'].quantile(0.2): return 1
    elif e < students['ques_quant'].quantile(0.3): return 2
    elif e < students['ques_quant'].quantile(0.4): return 3
    elif e < students['ques_quant'].quantile(0.5): return 4
    elif e < students['ques_quant'].quantile(0.6): return 5
    elif e < students['ques_quant'].quantile(0.7): return 6
    elif e < students['ques_quant'].quantile(0.8): return 7
    elif e < students['ques_quant'].quantile(0.9): return 8
    else: return 9
    
students['q_group'] = students['ques_quant'].apply(ques_convert)

In [ ]:
students.boxplot(column=['av_grade'], by=['q_group'])

# Мы видим странную просадку в 4м квартиле

In [ ]:
# Можем предположить, что первые вопросы являются сильным демотиватором для учащегося.
# Но если студент отвечает больше, чем на 34 вопроса, он постепенно начинает улучшать свои показатели
# И дальше результаты только улучшаются
# На гистограме видно, что 32 вопроса - порог отсечения для многих студентов из нашей выборки

students[(students['ques_quant'] < students['ques_quant'].quantile(0.5))].hist(column=['ques_quant'])

In [ ]:
"""
Построим диаграмму рассеяния, показывающую зависимость правильных ответов от кол-ва виденных объяснений
"""
students[(students['expl_watched'] < 500)].plot.scatter(x='av_grade', y='expl_watched', c='yellow')

In [ ]:
"""
Построим новый столбец с группировкой по кол-ву правильных ответов
Далее строим диаграмму размаха
"""
def expl_convert(e):
    if e < students['expl_watched'].quantile(0.1): return 0
    elif e < students['expl_watched'].quantile(0.2): return 1
    elif e < students['expl_watched'].quantile(0.3): return 2
    elif e < students['expl_watched'].quantile(0.4): return 3
    elif e < students['expl_watched'].quantile(0.5): return 4
    elif e < students['expl_watched'].quantile(0.6): return 5
    elif e < students['expl_watched'].quantile(0.7): return 6
    elif e < students['expl_watched'].quantile(0.8): return 7
    elif e < students['expl_watched'].quantile(0.9): return 8
    else: return 9
    
students['e_group'] = students['expl_watched'].apply(expl_convert)
students['expl_watched'].quantile(0.5)

In [ ]:
# здесь мы видим, что 34 вопроса - порог отсечения для многих студентов
students[(students['expl_watched'] < students['expl_watched'].quantile(0.8))].hist(column=['expl_watched'])

In [ ]:
students.boxplot(column=['av_grade'], by=['e_group'])

# Видно четкий тренд на повышение оценки с ростом кол-ва просмотренных объяснений

In [ ]:

"""
Построим диаграмму рассеяния, показывающую зависимость правильных ответов от кол-ва просмотренных лекций
"""
students[students['lec_watched'] < 21].plot.scatter(x='av_grade', y='lec_watched', c='green')

# Мы видим, что экстремальные выбросы можно убрать, ограничив число 21 лекцией

In [ ]:
"""
Построим диаграмму размаха, показывающую зависимость правильных ответов от кол-ва просмотренных лекций
"""
students[students['lec_watched'] < 21].boxplot(column=['av_grade'], by=['lec_watched'])

# Мы видим, что даже просмотр 1 леции повышает средний балл и значительно повышает минимальную оценку студента.
# Максимальный эффект достигается при просмотре 4-11 лекций, после чего эффективность просмотров снижается

In [ ]:
questions['question_id'].groupby(questions['bundle_id']).count().max()
questions['part'].value_counts(normalize=True)
questions['tags'].describe()

# Часть вопросов объединены по bundle_id в блоки до 5 вопросов, хотя большинство представлены по одиночке
# Задачи разделены по темам на 7 разделов, больше всего посвящено 5му разделу, 2, 3 и 4

In [ ]:
q_tags = set()
for tag in questions['tags']:
    try:
        for t in tag.split():
                q_tags.add(int(t))
    except:
        q_tags.add(int(t))
len(q_tags)

# Можно провести доп исследование вопросов, используя кластеризацию по 188 доп.признакам 'tags'

In [ ]:
"""
Для дальнейшего исследования вопросов создадим единую таблицу по их ID с данными Train
"""
tmp_df = train.loc[(train.content_type_id == 0), ['content_id', 'answered_correctly']]
"""
Столбец question_id:
"""
q_list = list(tmp_df['content_id'].unique())
len(q_list)

# Всего 13523 уникальных вопросов. Мы берем этот список из файла Train, чтобы иметь правильный порядок данных
13523
"""
Столбцы :
q_quant - общее количество ответов для каждого id
correct_quant - количество правильных ответов для каждого id
"""
q_quant = []
correct_quant = []
for q in q_list:
    tmp = tmp_df[tmp_df['content_id'] == q]['answered_correctly'].count()
    tmp1 = tmp_df[tmp_df['content_id'] == q]['answered_correctly'].sum()
    q_quant.append(tmp)
    correct_quant.append(tmp1)
"""
Запишем полученные данные в единую таблицу
"""
q_ex = pd.DataFrame({'question_id': q_list,
                        'q_quant': q_quant,
                        'correct_quant': correct_quant
                       }
                      )
q_ex = q_ex.astype({'question_id': 'int16', 'q_quant': 'int32', 'correct_quant': 'int32'})
"""
Внесем полученные данные в таблицу questions (пересечением),
вначале удалив ненужные столбцы
"""
questions = questions.drop('correct_answer', axis=1)
questions = pd.merge(questions, q_ex, how='inner')
"""
Добавим дополнительный столбец:
Процент правильных ответов
"""
questions['correct_percent'] = questions['correct_quant'] / questions['q_quant']
questions.describe()

In [ ]:
"""
Посмотрим дополнительные условия, от которых может зависеть успешность ответа на вопрос:
- от номера раздела в тесте TOEIC ('part')
"""
questions.groupby('part').mean()['correct_percent'].sort_values()

# По мере продвижения по разделам, видимо, сложность курса возрастает. 
# 5й раздел содержит максимальное число леций и заданий, однако средняя успеваемость студентов
# для него самая низкая.

In [ ]:
lectures = pd.read_csv('lectures.csv', sep=',',
                      dtype = {'lecture_id': 'int16', 'tag': 'int16', 
                               'part': 'int8', 'type_of': 'object'})
lectures.info()

In [ ]:
"""
Исследуем кол-во уникальных значений для разных колонок
"""
unique_list_lec = []
for col in lectures.columns:
    item = (col, lectures[col].nunique(), lectures[col].dtype)
    unique_list_lec.append(item)
unique_counts_lec = pd.DataFrame(unique_list_lec,
                                 columns=['Column_Name', 'Num_Unique', 'Type']
                                ).sort_values(by='Num_Unique',  ignore_index=True)
display(unique_counts_lec)

# Лекции разделены по темам на 7 разделов, больше всего лекций посвящено 5му разделу, затем 6, 2 и 1
# С учетом данных по вопросам, можно прийти к заключению, что 5 раздел - самый насыщенный по материалу,
# 6 и 1 - более теоретические, а 3 и 4 - более прикладные.
# Леции бывают 4 типов: вступление, целеполагание, концептуальное изложение материала и решение задач.
# Большинство лекций посвящены теории, немного меньше - решению задач. Доля остальных несущественна.
# Есть 151 доп.тип лекций, по которым можно провести кластеризацию

In [ ]:
lectures['part'].value_counts(normalize=True)
lectures['type_of'].value_counts(normalize=True)
lectures['tag'].value_counts().head()

In [ ]:
"""
Для дальнейшего исследования лекций создадим единую таблицу по их ID с данными Train.
"""
l_list = list(train[train['content_type_id'] == 1]['content_id'].unique())
len(l_list)

# Всего 415 уникальных лекций. Мы берем этот список из файла Train, чтобы иметь правильный порядок данных

In [ ]:
"""
Посчитаем количество просмотров лекций для каждого id
"""
l_quant = []
for l in l_list:
    tmp_l = train[(train['content_id'] == l)]['content_id'].count()
    l_quant.append(tmp_l)
"""
Запишем полученные данные в единую таблицу
"""
l_ex = pd.DataFrame({'lecture_id': l_list,
                        'l_quant': l_quant
                       }
                      )
l_ex = l_ex.astype({'lecture_id': 'int16', 'l_quant': 'int32'})
"""
Внесем полученные данные в таблицу lectures (пересечением)
"""
lectures = pd.merge(lectures, l_ex, how='inner')
display(lectures)

In [ ]:
"""
Общий рейтинг популярности для лекций
"""
lectures.sort_values(by=['l_quant'], ascending=False).head()

In [ ]:
"""
Рейтинг популярности для лекций, в зависимости от раздела
"""
lectures.groupby('part').sum()['l_quant'].sort_values(ascending=False)

In [ ]:
"""
Рейтинг относительной популярности лекций по разделам
"""
for part in range(1, 8):
    print(part, lectures.groupby('part').sum()['l_quant'].sort_values(ascending=False)[part]/lectures['part'].value_counts()[part])
    
# Наибольшее число просмотров имеет лекция из 7го раздела, 
# при этом самая высокая средневзвешенная популярность у лекций 2 раздела

In [ ]:
"""
Рейтинг популярности для лекций, в зависимости от тега лекции
"""
lectures.groupby('tag').sum()['l_quant'].sort_values(ascending=False).head()

In [ ]:
#Новая таблица
"""
Столбец part
"""
part_list = list(range(1, 8))
"""
Столбец кол-во вопросов в каждом разделе
"""
ques_quant_p = [questions[questions.part == p]['question_id'].count() for p in range(1,8)]
"""
Столбец кол-во ответов в каждом разделе
"""
ans_quant_p = [questions[questions.part == p]['q_quant'].sum() for p in range(1,8)]
"""
Столбец кол-во правильных ответов в каждом разделе
"""
right_quant_p = [questions[questions.part == p]['correct_quant'].sum() for p in range(1,8)]
"""
Столбец относительное кол-во правильных ответов в каждом разделе
"""
right_perc_p = [round(questions[questions.part == p]['correct_quant'].sum()/questions[questions.part == p]['q_quant'].sum(), 3) for p in range(1,8)]
"""
Столбец кол-во лекций в каждом разделе
"""
lec_quant_p = [lectures[lectures.part == p]['lecture_id'].count() for p in range(1,8)]
"""
Столбец кол-во просмотров лекций в каждом разделе
"""
lec_view_p = [lectures[lectures.part == p]['l_quant'].sum() for p in range(1,8)]
"""
Популярность лекций в каждом разделе
"""
lec_pop_p = [lectures[lectures.part == p]['l_quant'].sum()/lectures[lectures.part == p]['lecture_id'].count() for p in range(1,8)]
"""
Нормализованная популярность лекций в каждом разделе
"""
norm_lec_pop_p = [round(pop / max(lec_pop_p), 3) for pop in lec_pop_p]
"""
Запишем полученные данные в единую таблицу
"""
part_df = pd.DataFrame({'part': part_list,
                        'ques_quant_p': ques_quant_p,
                        'ans_quant_p': ans_quant_p,
                        'right_quant_p': right_quant_p,
                        'right_perc_p': right_perc_p,
                        'lec_quant_p': lec_quant_p,
                        'lec_view_p': lec_view_p,
                        'norm_lec_pop_p': norm_lec_pop_p
                       }
                      )
part_df = part_df.astype({'part': 'int8',
                        'ques_quant_p': 'int16',
                        'ans_quant_p': 'int64',
                        'right_quant_p': 'int64',
                        'right_perc_p': 'float32',
                        'lec_quant_p': 'int16',
                        'lec_view_p': 'int32',
                        'norm_lec_pop_p': 'float32'
                       })
part_df.sort_values(by='right_perc_p', ascending=False)